In [1]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [2]:
# Read the draft_2008-2018.csv file from the Resources folder into a Pandas DataFrame
draft_data_df = pd.read_csv(
    Path("./Resources/college/players.csv")
)

# Review the DataFrame
draft_data_df.head()

,ID,active_from,active_to,birth_date,college,height (in),name,position,weight,NBA__3ptapg,...,NCAA_ftpg,NCAA_games,NCAA_ppg,NBA Games Played,fg%,ppg,effective pct,height/weight,3pt %,Success
0,3,1991,2001,9-Mar-69,Louisiana State University,73,Mahmoud Abdul-Rauf,G,162,2.3,...,5.5,64,29.0,0.041619,0.0884,0.145515,0.062933,0.090123,0.0354,0.463991
1,5,1997,2008,11-Dec-76,University of California,81,Shareef Abdur-Rahim,F,225,0.6,...,6.1,28,21.1,0.058949,0.0944,0.180399,0.063867,0.072000,0.0297,0.499314
2,10,2006,2009,21-Jan-83,Pepperdine University,77,Alex Acker,G,185,0.8,...,2.9,88,14.5,0.002131,0.0740,0.026910,0.055067,0.083243,0.0320,0.273351
3,14,2013,2018,6-Oct-90,Baylor University,79,Quincy Acy,F,240,1.5,...,2.5,139,9.8,0.023082,0.0892,0.049834,0.068800,0.065833,0.0355,0.332250
4,19,2015,2016,8-Jul-94,"University of California, Los Angeles",77,Jordan Adams,G,209,0.8,...,4.2,69,16.4,0.002273,0.0804,0.031894,0.060933,0.073684,0.0385,0.287684


In [3]:
draft_data_df = draft_data_df.drop(columns=["name", "active_from", "active_to", "birth_date"])
draft_data_df.head()

,ID,college,height (in),position,weight,NBA__3ptapg,NBA__3ptpct,NBA__3ptpg,NBA_efgpct,NBA_fg%,...,NCAA_ftpg,NCAA_games,NCAA_ppg,NBA Games Played,fg%,ppg,effective pct,height/weight,3pt %,Success
0,3,Louisiana State University,73,G,162,2.3,0.354,0.8,0.472,0.442,...,5.5,64,29.0,0.041619,0.0884,0.145515,0.062933,0.090123,0.0354,0.463991
1,5,University of California,81,F,225,0.6,0.297,0.2,0.479,0.472,...,6.1,28,21.1,0.058949,0.0944,0.180399,0.063867,0.072000,0.0297,0.499314
2,10,Pepperdine University,77,G,185,0.8,0.320,0.3,0.413,0.370,...,2.9,88,14.5,0.002131,0.0740,0.026910,0.055067,0.083243,0.0320,0.273351
3,14,Baylor University,79,F,240,1.5,0.355,0.5,0.516,0.446,...,2.5,139,9.8,0.023082,0.0892,0.049834,0.068800,0.065833,0.0355,0.332250
4,19,"University of California, Los Angeles",77,G,209,0.8,0.385,0.3,0.457,0.402,...,4.2,69,16.4,0.002273,0.0804,0.031894,0.060933,0.073684,0.0385,0.287684


In [4]:
draft_data_df.dtypes

ID                    int64
college              object
height (in)           int64
position             object
weight                int64
NBA__3ptapg         float64
NBA__3ptpct         float64
NBA__3ptpg          float64
NBA_efgpct          float64
NBA_fg%             float64
NBA_fg_per_game     float64
NBA_fga_per_game    float64
NBA_ft%             float64
NBA_ft_per_g        float64
NBA_fta_p_g         float64
NBA_g_played          int64
NBA_ppg             float64
NCAA__3ptapg        float64
NCAA__3ptpct        float64
NCAA__3ptpg         float64
NCAA_efgpct         float64
NCAA_fgapg          float64
NCAA_fgpct          float64
NCAA_fgpg           float64
NCAA_ft             float64
NCAA_ftapg          float64
NCAA_ftpg           float64
NCAA_games            int64
NCAA_ppg            float64
NBA Games Played    float64
fg%                 float64
ppg                 float64
effective pct       float64
height/weight       float64
3pt %               float64
Success             

In [5]:
draft_data_df[["ID", "height (in)", "weight", "NBA_g_played", "NCAA_games"]] = draft_data_df[["ID", "height (in)", "weight", "NBA_g_played", "NCAA_games"]].astype(float)
draft_data_df.head()

,ID,college,height (in),position,weight,NBA__3ptapg,NBA__3ptpct,NBA__3ptpg,NBA_efgpct,NBA_fg%,...,NCAA_ftpg,NCAA_games,NCAA_ppg,NBA Games Played,fg%,ppg,effective pct,height/weight,3pt %,Success
0,3.0,Louisiana State University,73.0,G,162.0,2.3,0.354,0.8,0.472,0.442,...,5.5,64.0,29.0,0.041619,0.0884,0.145515,0.062933,0.090123,0.0354,0.463991
1,5.0,University of California,81.0,F,225.0,0.6,0.297,0.2,0.479,0.472,...,6.1,28.0,21.1,0.058949,0.0944,0.180399,0.063867,0.072000,0.0297,0.499314
2,10.0,Pepperdine University,77.0,G,185.0,0.8,0.320,0.3,0.413,0.370,...,2.9,88.0,14.5,0.002131,0.0740,0.026910,0.055067,0.083243,0.0320,0.273351
3,14.0,Baylor University,79.0,F,240.0,1.5,0.355,0.5,0.516,0.446,...,2.5,139.0,9.8,0.023082,0.0892,0.049834,0.068800,0.065833,0.0355,0.332250
4,19.0,"University of California, Los Angeles",77.0,G,209.0,0.8,0.385,0.3,0.457,0.402,...,4.2,69.0,16.4,0.002273,0.0804,0.031894,0.060933,0.073684,0.0385,0.287684


In [6]:
# Create a list of categorical variables 
categorical_variables = ["college", "position"]

# Display the categorical variables list
categorical_variables


['college', 'position']

In [7]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [8]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(draft_data_df[categorical_variables])

In [9]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns=enc.get_feature_names(categorical_variables)
)

# Review the DataFrame
encoded_df

,"college_Alcorn State University, Jackson State University",college_Arizona State University,college_Auburn University,"college_Auburn University, Florida State University","college_Auburn University, Georgia Institute of Technology",college_Austin Peay State University,college_Ball State University,"college_Ball State University, Oakland University",college_Baylor University,"college_Baylor University, Oklahoma State University",...,"college_Winston-Salem State University, Oral Roberts University",college_Wright State University,college_Xavier University,position_C,position_C-F,position_F,position_F-C,position_F-G,position_G,position_G-F
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1352,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [10]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
numerical_values_df = draft_data_df.drop(columns=categorical_variables)
encoded_df = pd.concat([encoded_df, numerical_values_df], axis=1)

# Review the Dataframe
encoded_df

,"college_Alcorn State University, Jackson State University",college_Arizona State University,college_Auburn University,"college_Auburn University, Florida State University","college_Auburn University, Georgia Institute of Technology",college_Austin Peay State University,college_Ball State University,"college_Ball State University, Oakland University",college_Baylor University,"college_Baylor University, Oklahoma State University",...,NCAA_ftpg,NCAA_games,NCAA_ppg,NBA Games Played,fg%,ppg,effective pct,height/weight,3pt %,Success
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.5,64.0,29.0,0.041619,0.0884,0.145515,0.062933,0.090123,0.0354,0.463991
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.1,28.0,21.1,0.058949,0.0944,0.180399,0.063867,0.072000,0.0297,0.499314
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.9,88.0,14.5,0.002131,0.0740,0.026910,0.055067,0.083243,0.0320,0.273351
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.5,139.0,9.8,0.023082,0.0892,0.049834,0.068800,0.065833,0.0355,0.332250
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.2,69.0,16.4,0.002273,0.0804,0.031894,0.060933,0.073684,0.0385,0.287684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.5,132.0,17.5,0.008949,0.0788,0.032890,0.058667,0.082222,0.0295,0.291028
1350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,96.0,15.5,0.050781,0.0836,0.113621,0.065733,0.075238,0.0376,0.426574
1351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.4,143.0,13.2,0.017685,0.0884,0.057807,0.061600,0.070909,0.0280,0.324401
1352,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.7,31.0,14.4,0.058239,0.0998,0.133555,0.068933,0.072398,0.0326,0.465525


In [38]:
encoded_df["NBA_ft%"].fillna()

,"college_Alcorn State University, Jackson State University",college_Arizona State University,college_Auburn University,"college_Auburn University, Florida State University","college_Auburn University, Georgia Institute of Technology",college_Austin Peay State University,college_Ball State University,"college_Ball State University, Oakland University",college_Baylor University,"college_Baylor University, Oklahoma State University","college_Baylor University, University of Nebraska",college_Belmont University,college_Boise State University,college_Boston College,"college_Boston College, California State University, Fresno",college_Boston University,college_Bowling Green State University,college_Bradley University,"college_Bradley University, University of New Mexico",college_Brigham Young University,college_Bucknell University,"college_Butler County Community College, Oklahoma State University",college_Butler University,"college_California Polytechnic State University, San Luis Obispo","college_California State University, Fresno","college_California State University, Fullerton","college_California State University, Long Beach","college_Cameron University, Southern University and A&M College",college_Centenary College of Louisiana,college_Central Michigan University,college_Clemson University,"college_Clemson University, University of Oklahoma",college_Cleveland State University,college_College of Charleston,college_College of the Holy Cross,college_Colorado State University,college_Creighton University,college_Davidson College,college_DePaul University,"college_DePaul University, University of Louisiana at Monroe","college_DePaul University, University of Texas at Austin",college_Delaware State University,"college_Delta State University, University of Mississippi",college_Drexel University,"college_Drexel University, University of Louisville",college_Duke University,"college_Duke University, Syracuse University","college_Duke University, University of Memphis",college_Duquesne University,"college_Duquesne University, University of Arizona",college_East Carolina University,college_East Tennessee State University,college_Eastern Illinois University,college_Eastern Michigan University,college_Eastern Washington University,college_Florida State University,college_Fordham University,college_George Washington University,college_Georgetown University,college_Georgia Institute of Technology,college_Georgia Southern University,college_Georgia State University,college_Gonzaga University,college_Hampton University,college_Harvard University,college_Hofstra University,"college_Howard College, Texas Tech University",college_Indiana University,"college_Indiana University, Georgetown University","college_Indiana University, Georgia Institute of Technology","college_Indiana University, Ohio State University","college_Indiana University, University of Houston","college_Indiana University, Xavier University",college_Indiana University-Purdue University Indianapolis,college_Iona College,college_Iowa State University,"college_Iowa State University, Louisiana State University","college_Iowa State University, Syracuse University","college_Iowa State University, Tyler Junior College, University of Houston",college_Jackson State University,college_Jacksonville State University,college_Jacksonville University,college_Kansas State University,college_La Salle University,"college_La Salle University, Towson University",college_Lamar University,"college_Lamar University, University of New Orleans",college_Lehigh University,"college_Liberty University, Duke University",college_Louisiana State University,college_Louisiana Tech University,college_Loyola College in Maryland,"college_Loyola Marymount University, University of California, Los Angeles","college_Loyola Marymount University, University of California, Santa Barbara",college_Loyola University of Chicago,college_Marquette University,college_Marshall University,"college_Marshall University, University

In [11]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df["Success"]

# Display a sample of y
y[:5]

0    0.463991
1    0.499314
2    0.273351
3    0.332250
4    0.287684
Name: Success, dtype: float64

In [12]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = encoded_df.drop(columns=["Success"])

# Review the features DataFrame
X[:5]

,"college_Alcorn State University, Jackson State University",college_Arizona State University,college_Auburn University,"college_Auburn University, Florida State University","college_Auburn University, Georgia Institute of Technology",college_Austin Peay State University,college_Ball State University,"college_Ball State University, Oakland University",college_Baylor University,"college_Baylor University, Oklahoma State University",...,NCAA_ftapg,NCAA_ftpg,NCAA_games,NCAA_ppg,NBA Games Played,fg%,ppg,effective pct,height/weight,3pt %
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.4,5.5,64.0,29.0,0.041619,0.0884,0.145515,0.062933,0.090123,0.0354
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.9,6.1,28.0,21.1,0.058949,0.0944,0.180399,0.063867,0.072000,0.0297
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.5,2.9,88.0,14.5,0.002131,0.0740,0.026910,0.055067,0.083243,0.0320
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,3.6,2.5,139.0,9.8,0.023082,0.0892,0.049834,0.068800,0.065833,0.0355
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,4.2,69.0,16.4,0.002273,0.0804,0.031894,0.060933,0.073684,0.0385


In [13]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [14]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

369

In [16]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [17]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  number_input_features

# Review the number hidden nodes in the first layer
hidden_nodes_layer1

369

In [18]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  (hidden_nodes_layer1 + 1) // 2

# Review the number hidden nodes in the second layer
hidden_nodes_layer2

185

In [19]:
# Create the Sequential model instance
nn = Sequential()

In [20]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

In [21]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

In [22]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="linear"))

In [23]:
# Compile the Sequential model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

In [24]:
# Fit the model using 50 epochs and the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
32/32 [==============================] - 0s 1ms/step - loss: nan - mse: nan
Epoch 2/50
32/32 [==============================] - 0s 1ms/step - loss: nan - mse: nan
Epoch 3/50
32/32 [==============================] - 0s 969us/step - loss: nan - mse: nan
Epoch 4/50
32/32 [==============================] - 0s 1ms/step - loss: nan - mse: nan
Epoch 5/50
32/32 [==============================] - 0s 1ms/step - loss: nan - mse: nan
Epoch 6/50
32/32 [==============================] - 0s 1ms/step - loss: nan - mse: nan
Epoch 7/50
32/32 [==============================] - 0s 1ms/step - loss: nan - mse: nan
Epoch 8/50
32/32 [==============================] - 0s 1ms/step - loss: nan - mse: nan
Epoch 9/50
32/32 [==============================] - 0s 1ms/step - loss: nan - mse: nan
Epoch 10/50
32/32 [==============================] - 0s 1ms/step - loss: nan - mse: nan
Epoch 11/50
32/32 [==============================] - 0s 1ms/step - loss: nan - mse: nan
Epoch 12/50
32/32 [====================